<center>
<img src="../../img/ods_stickers.jpg">
## Open Machine Learning Course
<center>
Author: Yury Kashnitsky, Data Scientist at Mail.Ru Group

This material is subject to the terms and conditions of the license [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Free use is permitted for any non-comercial purpose with an obligatory indication of the names of the authors and of the source.

## <center>Assignment #6. Part 1
### <center> Beating benchmarks in "Catch Me If You Can: Intruder Detection through Webpage Session Tracking"
    
[Competition](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2). The task is to beat "Assignment 6 baseline".

In [12]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer

Reading original data

In [7]:
PATH_TO_DATA = ('../../data')
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions_Alice.csv'), index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions_Alice.csv'), index_col='session_id')
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [15]:
'''блок, спизженный из базового ядра: 
# какая-то мутная переобработка столбцов time в тип datetime, в котором они и так уже находятся
и сортировка сессий по времени начала (что логично для 30-минутных сессий)
'''
# Switch time1, ..., time10 columns to datetime type
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# Sort the data by time
train_df = train_df.sort_values(by='time1')

'''обработка пропущенных данных по сайтам: в данном случае, если данных нет, значит,
их никогда и не было (не посетил пользователь аж 10 сайтов за 30 минут, хватило пары).
Поэтому смело выставляем время пребывания на сайтах, которые пользователь не посещал, равным 0'''
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

'''
Подгружаем словарь возможных сайтов.
Заводим под него dataframe (т.е., проиндексированную табличку в один столбец).
'''
# Load websites dictionary
with open(r"../../data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# Create dataframe for the dictionary
sites_dict = pd.DataFrame(list(site_dict.keys()), index=list(site_dict.values()), columns=['site'])
print(u'Websites total:', sites_dict.shape[0])
sites_dict.head()

#конец спизженного куска

Websites total: 48371


,site
44447,runway.blogs.nytimes.com
2458,calotag.com
4110,1.www.s81c.com
10687,www.brgm.fr
11880,www.u88.cn


Separate target feature 

In [16]:
y = train_df['target']

'''ещё один спизженный из базового ядра кусок - делаем данных для тренировки побольше,
для этого соединяем train без столбца target с testовой выборкой.
На всякий случай запоминаем место, где кончается train и начинается test, но как мы это
будем потом использовать - пока что непонятно
'''
# United dataframe of the initial data 
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# Index to split the training and test data sets
idx_split = train_df.shape[0]

In [25]:
print(full_df.shape)

(336358, 20)


Build Tf-Idf features based on sites. You can use `ngram_range`=(1, 3) and `max_features`=100000 or more

In [17]:
#отфильтруем сайты из общих данных
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


In [54]:
 print(full_sites.values.flatten())

[  56   55    0 ... 1098 1098 1098]


In [34]:
#кусок, спёртый из базового ядра
#берём массив идексов для сайтов
sites_flatten = full_sites.values.flatten()
#print(sites_flatten[:200])

#строим какую-то мутную матрицу, хрен пойми зачем. наверное, её и нужно будет заменить на Tf-Idf,
#если, конечно, соответствующая процедура тоже возвращает матрицу
# and the matrix we are looking for
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0]  + 10, 10)))[:, 1:]
print(full_sites_sparse.shape)
#матрица разреженная, очень большая. размерность 1 совпадает с числом сайтов в словаре, 
#размерность 0 - с числом записей в full_df (количество сессий) 
#т.е., у нас таблица сесси/сайты, 1 - сайт был просмотрен в сессии, 0 - не был

(336358, 48371)


In [37]:
sites_flatten1 = full_sites.to_string(index = False, header = False).split('\n')

In [42]:
print(type(sites_flatten1[1][1]))

<class 'str'>


In [45]:
#поробуем посмтроить Tf-Idf features
tf_idf_vect = TfidfVectorizer(ngram_range=(1, 3), max_features=100000, stop_words=['0'])
feats = tf_idf_vect.fit_transform(sites_flatten1)
print(feats)
#tf_idf_vect.fit_transform(sites_flatten)

  (0, 77307)	0.5679366365791909
  (0, 76129)	0.547845907916758
  (0, 77389)	0.6142579572215398
  (1, 77307)	0.44294963590589365
  (1, 76129)	0.42728031582168413
  (1, 77389)	0.4790769268600275
  (1, 76524)	0.27829124057086724
  (1, 77437)	0.4621621949074573
  (1, 76549)	0.3172893646694752
  (2, 98322)	0.7231309174833894
  (2, 98514)	0.14878156389677008
  (2, 98306)	0.16320479884930142
  (2, 98437)	0.11127646156746086
  (2, 90529)	0.08994258897039879
  (2, 98502)	0.1734002589164668
  (2, 98329)	0.331822790298673
  (2, 98341)	0.19232016818649045
  (2, 98518)	0.19596592529298001
  (2, 98328)	0.18754552308594596
  (2, 98312)	0.18594822958989943
  (2, 98462)	0.18930280056878676
  (2, 98504)	0.2068350906636841
  (2, 98342)	0.20954168696371916
  (3, 98322)	0.4343055245857754
  (3, 98306)	0.49009621954651084
  :	:
  (336355, 60643)	0.3547406212034198
  (336355, 34941)	0.35149865782625045
  (336356, 33155)	0.1362475609019598
  (336356, 25117)	0.1966016699710323
  (336356, 33579)	0.1061083140313

In [51]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
tfidf = transformer.fit_transform(full_sites_sparse)
print(tfidf)

  (0, 54)	0.6317732460448489
  (0, 55)	0.7751532529648281
  (1, 54)	0.6317732460448489
  (1, 55)	0.7751532529648281
  (2, 948)	0.2464534313966408
  (2, 783)	0.12065719748767129
  (2, 947)	0.14058385131142026
  (2, 944)	0.2202519125196865
  (2, 950)	0.18529867256157573
  (2, 945)	0.9066930156210695
  (3, 946)	0.20871999133515937
  (3, 945)	0.5696329513591194
  (3, 948)	0.258058668664328
  (3, 947)	0.29440759903027613
  (3, 944)	0.6918700420623607
  (4, 950)	0.20277555204892417
  (4, 945)	0.3968839587242487
  (4, 951)	0.26765351568866025
  (4, 947)	0.15384334741720343
  (4, 949)	0.5270828018704868
  (4, 946)	0.6544025815703585
  (5, 954)	0.2672522092651166
  (5, 945)	0.600736524351194
  (5, 952)	0.2422460905720338
  (5, 946)	0.660350082582811
  :	:
  (336354, 1215)	0.30327414266497954
  (336354, 1220)	0.30770776526618454
  (336354, 1217)	0.31665997506686333
  (336354, 1221)	0.3227464843766104
  (336354, 301)	0.5329120382860727
  (336354, 299)	0.5700058351005608
  (336355, 6779)	0.6294053

Add features based on the session start time: hour, whether it's morning, day or night and so on.

In [ ]:
# You code here

Scale this features and combine then with Tf-Idf based on sites (you'll need `scipy.sparse.hstack`)

In [ ]:
# You code here

Perform cross-validation with logistic regression.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

#скалер нужен для какого-то там масштабирования - не то модели, не то данных. 
#хрен его знает, судя по учебным материалам, хуже не будет
scaler = StandardScaler()
#завели логрегрессор
logit = LogisticRegression(random_state=17, class_weight='balanced')

logit_pipe = Pipeline([('scaler', scaler), ('logit', logit)])
logit_pipe_params = {'logit__C': np.logspace(-8, 8, 17)}

#кросс-валидация. какую брать не знаю, возьму такую для пробы
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

#завели валидатор по сетке
grid_search = GridSearchCV(logit_pipe, logit_pipe_params, cv=skf, scoring='roc_auc')
#собственно, запуск кросс-валидации, подбор параметров
grid_search.fit(X, y)

#печать лучших параметров и результатов при них
print(round(grid_search.best_score_, 3))
print(grid_search.best_index_)
print(grid_search.cv_results_['params'][grid_search.best_index_] )
print(grid_search.cv_results_['std_test_score'][grid_search.best_index_]/grid_search.best_score_)
print((grid_search.cv_results_['std_test_score'][grid_search.best_index_]/grid_search.best_score_ < 0.01))

Make prediction for the test set and form a submission file.

In [ ]:
test_pred = # You code here

In [ ]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)


In [ ]:
write_to_submission_file(test_pred, "assignment6_alice_submission.csv")